In [ ]:
import numpy as np
import sys

def generator_xor(x=50):
    input_data=[np.random.randint(2, size=2) for i in range(x)]
    labels=[[int(input_data[i][0]!=input_data[i][1])] for i in range(len(input_data))]
    return input_data, labels

def generator_even(x=100, size=25):
    input_data=[]
    for i in range(x):
        input_data.append(np.random.randint(2, size=size))
    labels=[[0,1] if np.sum(input_data[i])%2==1 else [1,0] for i in range(len(input_data))]
    return input_data, labels

def generator_half(x=100,size=25):
    input_data=[]
    for i in range(x):
        input_data.append(np.random.randint(2, size=size))
    labels=[[0] if np.sum(input_data[i])<size/2 else [1] for i in range(len(input_data))]
    return input_data, labels

class neural:
    def __init__(self,inputs,labels,layers=[3,24,12,1],acti=['sig','sig','sig','sig'], epsilon=.1, cost='mse',cycles=100):
        self.inputs=inputs
        self.input=self.inputs[0]
        self.labels=labels
        self.label=self.labels[0]
        self.layers=layers
        self.weights=[]
        self.into_nodes=[self.input]
        self.nodes_out=[self.input]
        self.acti=acti
        self.der_func=['der_'+i for i in self.acti]
        self.epsilon=epsilon
        self.cost=cost
        self.init=0
        self.cycles=cycles
    
    def neurons(self,weighted,func):
        if func=='sig':
            output=1/(1+np.exp(weighted*-1))
        elif func=='soft':#stable
            output=np.exp(weighted-np.max(weighted))/np.sum(np.exp(weighted-np.max(weighted)))
        elif func=="relu":
            output=[i if i>0 else i*0 for i in weighted]
        elif func=='der_sig':
            output=[1/(1+np.exp(weighted[i]*-1))*(1-1/(1+np.exp(weighted[i]*-1))) for i in range(len(weighted))]
        elif func=="der_soft":
            output=[i*(np.sum(weighted)-i)/np.sum(weighted)**2 for i in weighted]
        elif func=="der_relu":
            output=[1 if i>0 else i*0 for i in weighted]
        else:
            print('Unknown Function!')
        return np.array(output)
    
    def layer(self):
        for layer in range(len(self.layers)):
            size=self.layers[layer]
            if self.init==0:
                weight=(np.random.rand(size,len(self.input))-0.5)/0.5
                self.weights.append(weight)
                self.into_nodes.append(np.dot(self.weights[layer],self.input))
                self.input=self.neurons(self.into_nodes[layer+1],self.acti[layer])
                self.nodes_out.append(self.input)
            else:
                self.into_nodes[layer+1]=np.dot(self.weights[layer],self.input)
                self.input=self.neurons(self.into_nodes[layer+1],self.acti[layer])
                self.nodes_out[layer+1]=np.array(self.input)
    
    def rate_of_change(self,new):
        self.total_error=self.total_error+new
    
    def error(self):
        if len(self.label)!=len(self.nodes_out[-1]) or self.cost not in ['mse','centropy']:
            print('labels do not match')
        else:
            if self.cost=='mse':
                current_error=np.sum((self.label-self.nodes_out[-1])**2)/len(self.label)
                der_error=2*(self.label-self.nodes_out[-1])            
            elif self.cost=="centropy":
                current_error=-1*np.log(self.nodes_out[-1][np.argmax(self.label)]+0.000001)
                der_error=self.nodes_out[-1]-self.label+0.00001
            self.rate_of_change(current_error)
            if len(self.labels)==self.count:
                    sys.stdout.write('Total Error = {0}\r'.format(np.divide(self.total_error,self.count)))
                    sys.stdout.flush()
            return der_error
    
    def delta (self,layer):
        if layer==len(self.layers)-1:
            return self.error()*self.neurons(self.into_nodes[layer+1],self.der_func[layer])
        else:
            return self.neurons(self.into_nodes[layer+1], 
                                self.der_func[layer])*np.dot(self.deltas[len(self.layers)-2-layer], 
                                                             self.weights[layer+1])
        
    def backprop(self,mode='reverse'):
        self.deltas=[]
        for layer in reversed(range(len(self.layers))):
            self.deltas.append(self.delta(layer))
            self.weights[layer]=np.add(self.weights[layer],self.epsilon*(self.deltas[len(self.layers)-layer-1].reshape(-1,1)*self.nodes_out[layer]))
        self.init=1
        
    def run(self):
        for self.rounds in range(self.cycles):
            self.count=0
            self.total_error=0
            for next_input, next_label in zip(self.inputs,self.labels):            
                self.layer()
                self.count+=1
                self.backprop()
                self.input=next_input
                self.label=next_label
                
            
    def predict(self,guesswhat):
        self.input=guesswhat
        self.layer()
        return self.nodes_out[-1]
        
            
stuff,labels=generator_even(1000)
NN=neural(stuff,labels,[16,2],['sig', 'sig'],0.01,cost='mse',cycles=500)
NN.run()